In [ ]:
import numpy as np
import tensorflow as tf
import random

# Definir el entorno en una cuadrícula 5x5
# 0: celda vacía, 1: objetivo
grid = np.zeros((5, 5))
grid[4, 4] = 1

# Parámetros
learning_rate = 0.1
gamma = 0.9
epsilon = 1.0  # para la exploración
decay_factor = 0.99

# Inicializar la tabla Q con ceros
Q = np.zeros((25, 4))

# Modelo de TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, input_shape=(25,), activation='linear')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate), loss='mse')

# Entrenamiento
for episode in range(1000):
    state = (0, 0)
    done = False
    steps = 0  # Contador de pasos para este episodio

    while not done:
        steps += 1  # Incrementar el contador de pasos
        state_index = state[0] * 5 + state[1]
        if random.random() < epsilon:
            # Tomar acción aleatoria
            action = random.choice([0, 1, 2, 3])
        else:
            # Tomar la acción con el valor Q más alto
            action = np.argmax(Q[state_index])

        # Obtener nuevo estado y recompensa
        new_state = list(state)
        if action == 0:
            new_state[0] = max(0, state[0] - 1)
        elif action == 1:
            new_state[0] = min(4, state[0] + 1)
        elif action == 2:
            new_state[1] = max(0, state[1] - 1)
        elif action == 3:
            new_state[1] = min(4, state[1] + 1)

        reward = grid[new_state[0], new_state[1]]

        # Actualizar la tabla Q usando Q-learning
        new_state_index = new_state[0] * 5 + new_state[1]
        target = reward + gamma * np.max(Q[new_state_index])
        Q[state_index, action] = (1 - learning_rate) * Q[state_index, action] + learning_rate * target

        # Actualizar el modelo de TensorFlow
        target_f = Q[state_index]
        model.fit(np.identity(25)[state_index:state_index + 1], target_f.reshape(-1, 4), epochs=1, verbose=0)

        # Verificar si hemos llegado al objetivo
        if reward == 1:
            done = True
            print(f"Episodio {episode + 1} completado en {steps} pasos.")
        else:
            state = tuple(new_state)

    epsilon *= decay_factor

# Guardar el modelo
model.save("simple_grid_agent.h5")
